In [7]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# ChromeDriver'ın tam yolu
chromedriver_path = r"C:\Users\Monster\Desktop\UDEMY\chromedriver.exe"

# Kullanıcı ajanını değiştirme
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Chrome WebDriver Service objesi oluşturma
service = Service(chromedriver_path)

# WebDriver'ı başlatma
driver = webdriver.Chrome(service=service, options=options)

# Veri depolamak için bir liste
data = []

# Text temizleme fonksiyonları
def clean_room_text(text):
    if text:
        # "2 +\n 1" formatını "2+1" formatına çevirme
        return text.replace('\n', '').replace(' ', '').strip()
    return 'N/A'

def clean_age_text(text):
    if text:
        # "14\n at Age" formatını sadece "14" olarak temizleme
        return text.split('\n')[0].strip()
    return 'N/A'

# Sayfa boyunca gezme ve veri çekme
for page in range(1, 5):
    url = f'https://www.hepsiemlak.com/en/istanbul-satilik?sortDirection=ASC&sortField=PRICE&p31=1000000&p32=200000000&p33={page}'
    driver.get(url)
    
    # Sayfanın yüklenmesi için bekleme süresi
    time.sleep(10)
    
    # Sayfa kaynağını alma
    html = driver.page_source
    
    # BeautifulSoup ile sayfa kaynağını parse etme
    soup = BeautifulSoup(html, 'html.parser')
    
    # İlanları çekme - card-bottom-cage class'ına sahip div'leri bul
    listings = soup.find_all('div', class_='card-bottom-cage')
    
    # Her ilandan verileri çekme
    for listing in listings:
        # Başlık
        title_tag = listing.find('h3')
        title = title_tag.text.strip() if title_tag else 'N/A'
        
        # Oda sayısı - houseRoomCount class'ından
        room_tag = listing.find('span', class_='houseRoomCount')
        room = clean_room_text(room_tag.text) if room_tag else 'N/A'
        
        # Metrekare - squareMeter class'ından
        size_tag = listing.find('span', class_='squareMeter')
        size = size_tag.text.strip() if size_tag else 'N/A'
        
        # Bina yaşı - buildingAge class'ından
        age_tag = listing.find('span', class_='buildingAge')
        age = clean_age_text(age_tag.text) if age_tag else 'N/A'
        
        # Kat - floortype class'ından
        floor_tag = listing.find('span', class_='floortype')
        floor = floor_tag.text.strip() if floor_tag else 'N/A'
        
        # Konum - list-view-location class'ından
        location_tag = listing.find('span', class_='list-view-location')
        location = location_tag.text.strip() if location_tag else 'N/A'
        
        # For Sale/Rent status - left class içindeki span'dan
        status_tag = listing.find('span', class_='left')
        status = status_tag.text.strip() if status_tag else 'N/A'
        
        data.append({
            'status': status,
            'title': title,
            'location': location,
            'size': size,
            'room': room,
            'age': age,
            'floor': floor
        })

# WebDriver'ı kapatma
driver.quit()

# Verileri pandas DataFrame'e çevirme
df = pd.DataFrame(data)

# Excel dosyasına yazdırma
df.to_excel('hepsiemlak_listings.xlsx', index=False, engine='openpyxl')

print("Veriler başarıyla 'hepsiemlak_listings.xlsx' dosyasına kaydedildi.")


Veriler başarıyla 'hepsiemlak_listings.xlsx' dosyasına kaydedildi.


In [8]:
df

,status,title,location,size,room,age,floor
0,For Sale Apartment,50 Square Meters Apartment For Sale in Esenyur...,Istanbul / Esenyurt / Pınar Mah.,50 m²,Stüdyo,8,9. Floor
1,For Sale Apartment,98 Square Meters Apartment For Sale in Bağcıla...,Istanbul / Bağcılar / Sancaktepe Mah.,98 m²,1+1,35,Basement
2,For Sale Apartment,85 Square Meters Apartment For Sale in Esenyur...,Istanbul / Esenyurt / Mevlana Mah.,85 m²,1+1,4,2. Floor
3,For Sale Apartment,75 Square Meters Apartment For Sale in Esenyur...,Istanbul / Esenyurt / Güzelyurt Mah.,75 m²,1+1,5,2. Floor
4,For Sale Apartment,100 Square Meters Apartment For Sale in Beylik...,Istanbul / Beylikdüzü / Cumhuriyet Mah.,100 m²,2+1,5,2. Floor
...,...,...,...,...,...,...,...
71,For Sale Apartment,110 Square Meters Apartment For Sale in Esenyu...,Istanbul / Esenyurt / Sultaniye Mah.,110 m²,2+1,2,Garden Floor
72,For Sale Bungalow,26 Square Meters Bungalow For Sale in Ataşehir...,Istanbul / Ataşehir / Ferhatpaşa Mah.,26 m²,2+1,1,
73,For Sale Apartment,120 Square Meters Apartment For Sale in Esenyu...,Istanbul / Esenyurt / Fatih Mah.,120 m²,2+1,3,2. Floor
74,For Sale Apartment,110 Square Meters Apartment For Sale in Esenyu...,Istanbul / Esenyurt / Güzelyurt Mah.,110 m²,2+1,5,Garden Floor
